In [24]:
import os
import io
import time
import sqlite3

In [77]:
# creates db and populates with 1,000,000 dummy rows if non-zero num passed in
def pop_db(pop):
    if os.path.exists('/repostdb.sqlite'):
        os.remove('/repostdb.sqlite')

    conn = sqlite3.connect('./repostdb.sqlite')
    c = conn.cursor()
    c.execute('CREATE TABLE IF NOT EXISTS Posts (Date INT, Content TEXT, Url TEXT, Location TEXT, Author TEXT, Title TEXT);')
    if pop:
        for i in range(1000000):
            c.execute('INSERT INTO Posts (Date, Content, Url, Location, Author, Title) VALUES (?, ?, ?, ?, ?, ?);',
                (24102018, 'CONTENT', 'URL', 'LOCATION', 'AUTHOR', 'TITLE'))
    conn.commit()
    c.close()

In [78]:
# TEST MULTIPLE SINGLE-INSERTS
def mult_sing(num_insert, pop):
    pop_db(pop)

    start = time.time()
    conn = sqlite3.connect('./repostdb.sqlite')
    c = conn.cursor()

    for i in range(num_insert):
        c.execute('INSERT INTO Posts (Date, Content, Url, Location, Author, Title) VALUES (?, ?, ?, ?, ?, ?);',
            (24102018, 'CONTENT', 'URL', 'LOCATION', 'AUTHOR', 'TITLE'))

    conn.commit()
    c.close()
    return (time.time() - start)

In [79]:
# TEST SINGLE MULTIPLE-INSERT
def sing_mult(num_insert, pop):
    pop_db(pop)

    start = time.time()
    conn = sqlite3.connect('./repostdb.sqlite')
    c = conn.cursor()

    rows = []
    for i in range(num_insert):
        rows.append((24102018, 'CONTENT', 'URL', 'LOCATION', 'AUTHOR', 'TITLE'))

    values = ', '.join(map(str, rows))
    c.execute('INSERT INTO Posts (Date, Content, Url, Location, Author, Title) VALUES {}'.format(values))

    conn.commit()
    c.close()
    return (time.time() - start)

In [80]:
# TEST EXECUTEMANY
def exec_many(num_insert, pop):
    pop_db(pop)

    start = time.time()
    conn = sqlite3.connect('./repostdb.sqlite')
    c = conn.cursor()

    rows = []
    for i in range(num_insert):
        rows.append((24102018, 'CONTENT', 'URL', 'LOCATION', 'AUTHOR', 'TITLE'))
    c.executemany("INSERT INTO Posts (Date, Content, Url, Location, Author, Title) VALUES (?, ?, ?, ?, ?, ?)", rows)

    conn.commit()
    c.close()
    return (time.time() - start)    

In [81]:
def test_avg(func, test_iters, pop):
    results = []
    for i in range(50):
        results.append(func(test_iters, pop))
    return sum(results)/len(results)

In [82]:
print("MULTIPLE SINGLE-INSERTS (EMPTY) ELAPSED TIME:     ", test_avg(mult_sing, 10000, False))
print("SINGLE MULTIPLE-INSERT (EMPTY) ELAPSED TIME:      ", test_avg(sing_mult, 10000, False))
print("EXECUTEMANY (EMPTY) ELAPSED  TIME:                ", test_avg(exec_many, 10000, False))
print("MULTIPLE SINGLE-INSERTS (POPULATED) ELAPSED TIME: ", test_avg(mult_sing, 10000, True))
print("SINGLE MULTIPLE-INSERT (POPULATED) ELAPSED TIME:  ", test_avg(sing_mult, 10000, True))
print("EXECUTEMANY (POPULATED) ELAPSED TIME:             ", test_avg(exec_many, 10000, True))

MULTIPLE SINGLE-INSERTS (EMPTY) ELAPSED TIME:      0.0352266263961792
SINGLE MULTIPLE-INSERT (EMPTY) ELAPSED TIME:       0.08103799819946289
EXECUTEMANY (EMPTY) ELAPSED  TIME:                 0.023240842819213868
MULTIPLE SINGLE-INSERTS (POPULATED) ELAPSED TIME:  0.036230764389038085
SINGLE MULTIPLE-INSERT (POPULATED) ELAPSED TIME:   0.08659755706787109
EXECUTEMANY (POPULATED) ELAPSED TIME:              0.023484020233154296
